In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
import glob
from PIL import Image

MODEL_NAME = 'faster_rcnn_inception_v2_coco_sim'

In [2]:
PATH_TO_CKPT = '../ros/src/tl_detector/model/' + MODEL_NAME + '/frozen_inference_graph.pb'

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

class TLClassifier(object):
    def __init__(self):
        detection_graph = TLClassifier.load_graph()
        self.detection_graph = detection_graph
        self.sess = tf.Session(graph=detection_graph)
        self.image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        self.detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        self.detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = detection_graph.get_tensor_by_name('num_detections:0')


    @staticmethod
    def load_graph():
        detection_graph = tf.Graph()
        print("Loading graph...")
        with detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
        print("Graph loaded!")
        return detection_graph

    def get_classification(self, image):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)

        # Actual detection.
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})

        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes).astype(np.int32)

        if len(scores) < 1 or scores[0] < 0.4:
            return "UNKNOWN";

        cls = classes[0]
        print(cls, scores[0])
        
        box = boxes[0]
        box_h = box[2] - box[0]
        box_w = box[3] - box[1]
        
        if box_h < 0.05 or box_w < 0.02:  
            print("too small")
            return "UNKNOWN"

        print('bounding box: {} {}'.format(box_h, box_w))
        
        if cls == 1:
            return "RED"
        elif cls == 2:
            return "YELLOW"
        elif cls == 3:
            return "GREEN"

        return "UNKNOWN"

        
PATH_TO_TEST_IMAGES_DIR = 'test_images_sim'
CLASSES = ['red', 'yellow', 'green', 'unknown']

paths = []
for cls in CLASSES:
    paths += glob.glob(PATH_TO_TEST_IMAGES_DIR + "/" + cls + "/*.jpg")


tl = TLClassifier()

for image_path in paths:
    image = Image.open(image_path)
    image_np = load_image_into_numpy_array(image)
    print(tl.get_classification(image_np))


Loading graph...
Graph loaded!
(3, 0.99895006)
bounding box: 0.218234032393 0.07377576828
GREEN
(2, 0.98773175)
bounding box: 0.227216541767 0.0763422399759
YELLOW
(1, 0.99992704)
bounding box: 0.240091532469 0.0840641856194
RED
UNKNOWN
